### Exports for FORGENIUS

In [ ]:
!pip install geemap

In [ ]:
!earthengine authenticate

Import relevant python modules

In [ ]:
import ee
ee.Initialize()
import geemap
Map = geemap.Map()
from IPython.display import JSON

Load GCU polygons and declare globals

In [ ]:
# load forgenius polygons
polygons = ee.FeatureCollection('projects/ee-milko/assets/gcu_shapes')

# globals
###
# Note that we now separate GCU information from shapes,
# so only the GeometryID will be used as unique identifier
# and metadata in properties.
###
export_folder = 'ForgeniusExport'
shape_id = 'GeometryID'
unit_id = 'UnitID'
gcu_id = 'UnitNumber'

###
# Europe
# Note that it includes Georgia, the Azores Islands and the Canary Islands.
###
europe = ee.Geometry.MultiPolygon(
  [
    [
			[
				[
					-25.695308647197542,
					67.23801112519807
				],
				[
					-27.14550395969755,
					63.898673465352644
				],
				[
					-20.333980522197542,
					62.22693529585702
				],
				[
					-10.138668022197548,
					63.898673465352644
				],
				[
					-10.885738334697546,
					67.1528475956907
				],
				[
					-19.718746147197546,
					67.79259157662841
				],
				[
					-25.695308647197542,
					67.23801112519807
				]
			]
		],
		[
			[
				[
					25.544925727802454,
					73.02255350915792
				],
				[
					12.31738666530245,
					71.41313528514112
				],
				[
					-10.395820973315923,
					63.1549422894266
				],
				[
					-14.445308647197546,
					56.92092561250376
				],
				[
					-11.918453178447548,
					27.039440497919774
				],
				[
					-8.754390678447548,
					25.562147421034734
				],
				[
					34.75146869655245,
					25.997432756878307
				],
				[
					51.73633197780245,
					37.857404237615626
				],
				[
					51.07024040994158,
					40.78364672467574
				],
				[
					40.87845938095193,
					49.75279571510055
				],
				[
					32.22123281845194,
					60.0428385062494
				],
				[
					34.72611563095193,
					72.47523705944548
				],
				[
					25.544925727802454,
					73.02255350915792
				]
			]
		],
		[
			[
				[
					-10.625446869048067,
					71.30075112528307
				],
				[
					-10.361774994048067,
					70.27424567977174
				],
				[
					-5.5717359315480675,
					70.67083792416264
				],
				[
					-6.7143140565480675,
					71.56660006121781
				],
				[
					-10.625446869048067,
					71.30075112528307
				]
			]
		],
		[
			[
				[
					-31.06358097506254,
					41.1772146823826
				],
				[
					-33.48057316256254,
					38.443486969032314
				],
				[
					-17.00108097506254,
					31.23935040308769
				],
				[
					-14.54014347506254,
					33.28302140035009
				],
				[
					-24.164166912562543,
					40.00932248815723
				],
				[
					-31.06358097506254,
					41.1772146823826
				]
			]
		],
		[
			[
				[
					-19.42731573478265,
					29.44294411691417
				],
				[
					-19.25153448478265,
					26.17860766874929
				],
				[
					-14.94489385978265,
					26.572313496293894
				],
				[
					-13.494698547282649,
					28.055982113199935
				],
				[
					-12.57184698478265,
					29.481206483971427
				],
				[
					-16.08747198478265,
					30.129448042603595
				],
				[
					-19.42731573478265,
					29.44294411691417
				]
			]
		]
  ]
)

Declare common functions

In [ ]:
###
# Export a descriptor from a feature collection in CSV format on Goofle Drive.
#
# filename:      Output variable name and filename.
# regions:      Features collection.
# selectors:    Output columns.
###
def exportFeatureCollection(filename, regions, selectors):
  task = ee.batch.Export.table.toDrive(collection=regions, folder=export_folder, description=filename, fileFormat='CSV', selectors=selectors)
  task.start()

###
# Filter high quality pixels,
# select required band,
# copy timestamp property to result.
#
# image:        Image to filter.
# qualityBand:  Name of QA band.
# dataBand:     Name of data band of interest.
###
def selectQualityPixels(image, mask, qualityBand, dataBand):
    # Select image quality band
    qa = image.select(qualityBand)

    # Create mask to filter pixels containing clouds
    cloudBitMask = mask
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)

    return image \
            .updateMask(mask) \
            .select(dataBand) \
            .copyProperties(image, ['system:time_start'])

###
# Aggregate data at monthly resolution.
#
# year:         Year to aggregate.
# multiplier:   Coefficient applied to all values.
###
def groupByMonth(collection, year, multiplier=1):
    months = ee.List.sequence(1, 12, 1)

    # Aggregate data by month.
    def aggregateByMonth(m):

      # Convert month to number.
      month = ee.Number(m)

      # Filter items in month,
      # extract values mean.
      monthly = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                    .filter(ee.Filter.calendarRange(month, month, 'month')) \
                    .mean()

      # Multiply result by coefficient, if latter is not 1.
      if multiplier != 1:
        monthly = monthly.multiply(multiplier)

      # Return image with aggregated value, year, month and timestamp.
      return monthly \
          .set('year', year) \
          .set('month', m) \
          .set('system:time_start', ee.Date.fromYMD(year, m, 1).millis())

    return months.map(aggregateByMonth)

###
# Get mean by month for GCU polygons.
#
# image:        Image to process.
# descriptor:   Feature name.
# scale:        Resolution scale in meters.
###
def monthlyRegions(image, descriptor, scale):

  def setMonthYear(feature):

    # Get year and month.
    year = ee.Image(image).get('year')
    month = ee.Image(image).get('month')

    # Return date feature.
    return feature.set({ 'year':year, 'month':month })

  # Return regions.
  return image.select(descriptor).reduceRegions(collection=polygons, reducer=ee.Reducer.mean().setOutputs([descriptor]), scale=scale).map(setMonthYear)

###
# Get mean by month for GCU polygons and mark unavailable values.
#
# image:        Image to process.
# descriptor:   Band and feature name.
# na:           Value for missing elements.
# scale:        Resolution scale in meters.
# projection:   Projection.
###
def zonalStatisticsNa(image, descriptor, na, scale, projection):

  def set_NA(feature):

    # Get list of non-null descriptor values.
    values = ee.List([feature.get(descriptor), na]).reduce(ee.Reducer.firstNonNull())

    # Set time frame.
    system_time_start = ee.Image(image).get('system:time_start')
    year = ee.Image(image).get('year')
    month = ee.Image(image).get('month')

    # Set metadata and return feature.
    return feature.set({descriptor: values, 'imageID': image.id(), 'system:time_start': system_time_start, 'year': year, 'month': month})

  return image.reduceRegions(collection=polygons, reducer=ee.Reducer.mean().setOutputs([descriptor]), scale=scale, crs=projection).map(set_NA)

# Merge image bands.
def mergeImageBands(joinResult):

  return ee.Image(joinResult.get('primary')).addBands(joinResult.get('secondary'))

# Wrapper function for masking clouds.
# Note: this function is not used.
def maskClouds(img):

  cloudProbabilityThreshold = 40
  cloudThresh = 40
  cloudMask = img.select('probability').lt(cloudProbabilityThreshold)
  img = s2CloudScore(img)

  return img.updateMask(cloudMask).updateMask(img.select('cloudScore').lt(cloudThresh))


### Topography

In [ ]:
# Load DEM
# https://www.eea.europa.eu/data-and-maps/data/copernicus-land-monitoring-service-eu-dem
# Version 1.0 or version 1.1?
# 40.56GB
model = ee.ImageCollection("projects/phenology-303215/assets/eu_dem") \
          .filterBounds(europe)

In [ ]:
# elevation
descriptor = 'elevation'
descriptor_mean = 'elevation_mean'
descriptor_stDev = 'elevation_stdDev'
# Create combined reducer to get mean and standard deviation.
reducer = ee.Reducer.mean().combine(reducer2=ee.Reducer.stdDev(), sharedInputs=True).setOutputs([descriptor_mean, descriptor_stDev])
regions = model.median().reduceRegions(collection=polygons, reducer=reducer, scale=25)
exportFeatureCollection(filename=descriptor, regions=regions, selectors=[shape_id, descriptor_mean, descriptor_stDev])

In [ ]:
# slope
descriptor = 'slope'
regions = model.map(lambda image: ee.Terrain.slope(image)).median().reduceRegions(collection=polygons, reducer=ee.Reducer.mean().setOutputs([descriptor]), scale=25)
exportFeatureCollection(filename=descriptor, regions=regions, selectors=[shape_id, descriptor])

In [ ]:
# aspect
descriptor = 'aspect'
regions = model.map(lambda image: ee.Terrain.aspect(image)).median().reduceRegions(collection=polygons, reducer=ee.Reducer.mean().setOutputs([descriptor]), scale=25)
exportFeatureCollection(filename=descriptor, regions=regions, selectors=[shape_id, descriptor])

### Canopy height

In [ ]:
# Set globals
year = 2019
descriptor = 'canopy_height'
filename = 'canopy_height_2019'

# load collection
# https://glad.umd.edu/dataset/gedi
model = ee.ImageCollection('users/potapovpeter/GEDI_V27') \
          .filterBounds(europe) \
          .mosaic()

# add date to feature polygons
collection = polygons.map(lambda feature: feature.set({ 'year': year }))

# apply reducer
regions = model.reduceRegions(collection=collection, reducer=ee.Reducer.mean().setOutputs([descriptor]), scale = 30)

# export results
exportFeatureCollection(filename=filename, regions=regions, selectors=[shape_id, 'year', descriptor])

### Dominant leaf type
https://code.earthengine.google.com/96720d5eac7e9db1de8e87c8cd7dd162

https://land.copernicus.eu/pan-european/high-resolution-layers/forests/dominant-leaf-type/status-maps/dominant-leaf-type-2018

In [ ]:
# Set globals
year = 2018
descriptor = 'dominant_forest_type'
filename = 'dominant_leaf_type'

# Load image and skip empty pixels.
model = ee.Image('users/marcogirardello/forgenius/domleaftypeeu')
model = model.updateMask(model.gt(0))

# add date to feature polygons
collection = polygons.map(lambda feature: feature.set({ 'year': year }))

# apply reducer (mode)
regions = model.reduceRegions(collection=collection, reducer=ee.Reducer.mode().setOutputs([descriptor]), scale=12, crs='EPSG:4326')

# export results
exportFeatureCollection(filename=descriptor, regions=regions, selectors=[shape_id, 'year', descriptor])

### Biomass for the year 2010
https://climate.esa.int/en/projects/biomass/data/

In [ ]:
# Set globals
year = 2010
descriptor = 'biomass_2010'
filename = 'biomass_2010'

# load collection
# https://climate.esa.int/en/projects/biomass/data/
model = ee.Image("users/guidolavespa2511/EU_biomass")

# add date to feature polygons
collection = polygons.map(lambda feature: feature.set({ 'year': year }))

# apply reducer
regions = model.reduceRegions(collection=collection, reducer=ee.Reducer.mean().setOutputs([descriptor]), scale = 98.95114197446169, crs='EPSG:4326')

# export results
exportFeatureCollection(filename=descriptor, regions=regions, selectors=[shape_id, 'year', descriptor])

## Monthly Day Land surface temperature (LST). Data are from terra, meaning that the satellite passes in the morning.
#### Catalogue entry https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A2

In [ ]:
# Set references
descriptor = 'LST_Day_1km'
startDate = '2000-01-01'
endDate = '2023-12-31'
dataBand = 'LST_Day_1km'
qualityBand = 'QC_Day'

# Filter image collection by:
# selecting Europe bounds,
# selecting years 2000 to 2021,
# select quality and data bands,
# select only good quality pixels.
collection = ee.ImageCollection("MODIS/061/MOD11A2") \
        .filterBounds(europe) \
        .filter(ee.Filter.date(startDate, endDate)) \
        .select([dataBand, qualityBand]) \
        .map(lambda image: selectQualityPixels(image=image, mask=(0 << 0), qualityBand=qualityBand, dataBand=dataBand))

# Aggregate data by year and month.
years = ee.List.sequence(2001, 2021, 1)
monthlyCollection = ee.ImageCollection.fromImages(years.map(lambda year: groupByMonth(collection=collection, year=year, multiplier=0.02)).flatten())

# Apply zonal statistics reducer.
regions = monthlyCollection.map(lambda image: zonalStatisticsNa(image=image, descriptor=descriptor, na=-9999, scale=926.6254330555, projection='SR-ORG:6974')).flatten()

# export results
exportFeatureCollection(filename=descriptor, regions=regions, selectors=[shape_id, 'year', 'month', descriptor])

### Leaf Area Index (LAI) (monthly)

#### Catalogue entry https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD15A2H?hl=en


In [ ]:
# Set references
descriptor = 'Lai_500m'
startDate = '2003-01-01'
endDate = '2023-12-31'
dataBand = 'Lai_500m'
qualityBand = 'FparLai_QC'

# Filter image collection by:
# selecting Europe bounds,
# selecting years 2000 to 2021,
# select quality and data bands,
# select only good quality pixels.
#https://gis.stackexchange.com/questions/376972/time-series-extraction-of-modis-lai-product-every-3-days-a-new-satellite-image
collection = ee.ImageCollection("MODIS/061/MOD15A2H") \
        .filterBounds(europe) \
        .filter(ee.Filter.date(startDate, endDate)) \
        .select([dataBand, qualityBand]) \
        .map(lambda image: selectQualityPixels(image=image, mask=(1 << 0), qualityBand=qualityBand, dataBand=dataBand))

# Aggregate data by year and month.
years = ee.List.sequence(2003, 2021, 1)
monthlyCollection = ee.ImageCollection.fromImages(years.map(lambda year: groupByMonth(collection=collection, year=year, multiplier=0.1)).flatten())

# check this bit as it does not work. year need changing (2024)
#test1 = monthlyCollection.filter(ee.Filter.listContains('system:band_names','Lai_500m'))

# Apply zonal statistics reducer.
regions = monthlyCollection.map(lambda image: zonalStatisticsNa(image=image, descriptor=descriptor, na=-9999, scale=463.3127165275, projection='SR-ORG:6974')).flatten()

# export results
exportFeatureCollection(filename=descriptor, regions=regions, selectors=[shape_id, 'year', 'month', descriptor])

### GPP (monthly)
#### Catalogue entry https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MYD17A2H?hl=en

In [ ]:
# Set references
descriptor = 'Gpp'
start_year = 2021
# startDate = '2003-01-01'
startDate = '2021-01-01'
end_year = 2023
# endDate = '2021-12-31'
endDate = '2023-08-05'
dataBand = 'Gpp'
qualityBand = 'Psn_QC'

# Filter image collection by:
# selecting Europe bounds,
# selecting years 2000 to 2021,
# select quality and data bands,
# select only good quality pixels.
collection = ee.ImageCollection("MODIS/061/MYD17A2H") \
        .filterBounds(europe) \
        .filter(ee.Filter.date(startDate, endDate)) \
        .select([dataBand, qualityBand]) \
        .map(lambda image: selectQualityPixels(image=image, mask=(1 << 0), qualityBand=qualityBand, dataBand=dataBand))

# Aggregate data by year and month.
years = ee.List.sequence(start_year, end_year, 1)
monthlyCollection = ee.ImageCollection.fromImages(years.map(lambda year: groupByMonth(collection=collection, year=year, multiplier=0.0001)).flatten())

# Apply zonal statistics reducer.
regions = monthlyCollection.map(lambda image: zonalStatisticsNa(image=image, descriptor=descriptor, na=-9999, scale=463.3127165275, projection='SR-ORG:6974')).flatten()

# export results
exportFeatureCollection(filename=descriptor, regions=regions, selectors=[shape_id, 'year', 'month', descriptor])

### NDVI from Sentinel 2

#### Guido's original code https://code.earthengine.google.com/8efacef6da5c9cfcfffcecfb0027190c

In [ ]:
# Set references
descriptor = 'ndvi'
filename = 'ndvi'
startDate = '2016-01-01'
endDate = '2021-12-31'
originalBandNames = ['QA60', 'B1','B2','B3','B4','B5','B6','B7','B8','B8A', 'B9','B10', 'B11','B12','probability']
renamedBandNames  = ['QA60','cb', 'blue', 'green', 'red', 're1','re2','re3','nir', 'nir2', 'waterVapor', 'cirrus','swir1', 'swir2','probability']

In [ ]:
# Local functions

# function for calculating and adding NDVI to the provided image.
def s2AddNDVI(img):

  ndvi = img.normalizedDifference(['B8','B4']).multiply(1000).rename(descriptor)

  return img.addBands(ndvi)

def rescale(img, exp, thresholds):

  return img.expression(exp, {'img': img}).subtract(thresholds[0]).divide(thresholds[1] - thresholds[0])

# function for cloudscoring
def s2CloudScore(img):

  # Compute several indicators of cloudyness and take the minimum of them.
  score = ee.Image(1)

  # Clouds are reasonably bright in the blue and cirrus bands.
  score = score.min(rescale(img, 'img.blue', [0.1, 0.5]))
  score = score.min(rescale(img, 'img.cb', [0.1, 0.3]))
  score = score.min(rescale(img, 'img.cb + img.cirrus', [0.15, 0.2]))

  # Clouds are reasonably bright in all visible bands.
  score = score.min(rescale(img, 'img.red + img.green + img.blue', [0.2, 0.8]))

  # Clouds are moist
  ndmi = img.normalizedDifference(['nir','swir1'])
  score=score.min(rescale(ndmi, 'img', [-0.1, 0.1]))

  # However, clouds are not snow
  ndsi = img.normalizedDifference(['green', 'swir1'])
  score=score.min(rescale(ndsi, 'img', [0.8, 0.6]))
  score = score.multiply(100).byte()

  return img.addBands(score.rename('cloudScore'))

# function to mask clouds using the Sentinel-2 QA band
def s2MaskClouds(img):
  qa = img.select('QA60').int16()

  # bits 10 and 11 are cloud and cirrus, respectively
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # both flags should be set to zero, indicating clear conditions
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask))

  # return the masked and scaled data
  return img.updateMask(mask).copyProperties(img,['system:time_start'])

# Divide all values by coefficient,
# add cloud probability bands
# and add image timestamp.
def s2CleanImage(img):

  # rescale 0-1
  t = img.select([ 'B1','B2','B3','B4','B5','B6','B7', 'B8','B8A', 'B9','B10', 'B11','B12']).divide(10000)

  # add QA60 and probability bands
  t = t.addBands(img.select(['QA60', 'probability']))
  out = t.copyProperties(img).copyProperties(img, ['system:time_start'])

  return out

In [ ]:
# Load Sentinel 2 data and clouds
sentinelData = ee.ImageCollection('COPERNICUS/S2').filterBounds(europe).filterDate(startDate, endDate)
sentinelCloud = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY").filterBounds(europe).filterDate(startDate, endDate)

# Join data and clouds by image index.
innerjoined = ee.Join.inner() \
             .apply(primary = sentinelData,
                    secondary = sentinelCloud,
                    condition = ee.Filter.equals(leftField = 'system:index',
                                                  rightField = 'system:index'))

# Merge joined collections.
newCollection = ee.ImageCollection(innerjoined.map(mergeImageBands))

# Rename bands.
newCollection1 = newCollection.map(s2CleanImage).select(originalBandNames, renamedBandNames)

# Filter cloudy images and restore original band names.
newCollection2 = newCollection1.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80)) \
              .map(s2MaskClouds) \
              .select( renamedBandNames, originalBandNames) \
              .map(s2AddNDVI) \
              .select(descriptor)

# aggregate to a monthly resolution
years = ee.List.sequence(2016, 2021, 1)
monthlyCollection = ee.ImageCollection.fromImages(years.map(lambda year: groupByMonth(collection=newCollection2, year=year)).flatten())

# Apply zonal statistics reducer.
regions = monthlyCollection.map(lambda image: monthlyRegions(image=image, descriptor=descriptor, scale=10)).flatten()

# export results
exportFeatureCollection(filename=filename, regions=regions, selectors=[shape_id, 'year', 'month', descriptor])

### NDWI
#### Catalogue entry https://developers.google.com/earth-engine/datasets/catalog/MODIS_MCD43A4_006_NDWI

In [ ]:
# Set references
descriptor = 'NDWI'
startDate = '2001-01-01'
endDate = '2023-12-31'

# Filter image collection by:
# selecting Europe bounds,
# selecting years 2001 to 2021,
# select quality and data bands,
# select only good quality pixels.
collection = ee.ImageCollection("MODIS/MCD43A4_006_NDWI") \
        .filterBounds(europe) \
        .filter(ee.Filter.date(startDate, endDate))

# Aggregate data by year and month.
years = ee.List.sequence(2003, 2021, 1)
monthlyCollection = ee.ImageCollection.fromImages(years.map(lambda year: groupByMonth(collection=collection, year=year)).flatten())

# Apply zonal statistics reducer.
regions = monthlyCollection.map(lambda image: zonalStatisticsNa(image=image, descriptor=descriptor, na=-9999, scale=463.3127165275, projection='SR-ORG:6974')).flatten()

# export results
exportFeatureCollection(filename=descriptor, regions=regions, selectors=[shape_id, 'year', 'month', descriptor])

### Land cover 10 m
#### Data source: https://www.esa.int/ESA_Multimedia/Images/2020/03/Europe_land-cover_mapped_in_10_m_resolution

Skip this one!

In [ ]:
#eulandcover = ee.Image('users/marcogirardello12/forgenius/eulandcover10m')

### ERA5 Land

In [ ]:
# Load ERA5 image collection.
era5Collection = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterBounds(europe)


###
# Get mean by day for GCU polygons.
#
# image:        Image to process.
# scale:        Resolution scale in meters.
# projection:   Image and geometry projection.
###
def dailyRegions(image, descriptor, scale, projection):

  # Aggregate means of GCUs and add year and day to features.
   return image.reduceRegions(collection=polygons, reducer=ee.Reducer.mean().setOutputs([descriptor]), scale=scale, crs=projection) \
          .map(lambda feature: feature.set({ 'year': image.get('year'), 'day': image.get('day') }))

###
# Calculate wind speed.
# U component is positive for a west to east flow (eastward wind)
# and the V component is positive for south to north flow (northward wind).
###
def windSpeed(image):

  # Select eastward and northward components and return wind speed and copy timestamp.
  return image.select(['u_component_of_wind_10m', 'v_component_of_wind_10m']) \
          .pow(2) \
          .reduce(ee.Reducer.sum()) \
          .sqrt() \
          .copyProperties(image, ['system:time_start'])

###
# Calculate relative humidity.
# Calculate using dew point.
###
def relativeHumidity(img):
  img1 = img.subtract(273.15)
  rh = ee.Image(100)\
          .subtract(img1.select('temperature_2m').subtract(img1.select('dewpoint_temperature_2m')) \
          .multiply(5)) \
          .select(['constant'],['relative_humidity']) \
          .copyProperties(img,['system:time_start'])
  return(rh)

###
# Calculate daily statistics,
###
def statsByDay(descriptor, band, prefix, startYear, endYear, scale, projection):

  # Iterate time interval years.
  for i in range(startYear, endYear):

    # Select start and end string dates of the focal year,
    fyear_start = ee.Date(str(i) + '-01-01')
    fyear_end = ee.Date(str(i) + '-12-31')

    # select band
    data = era5Collection.select(band).filter(ee.Filter.date(fyear_start, fyear_end))

    # days for focal year (based on collection size)
    days = ee.List.sequence(start=1, end=fyear_end.difference(fyear_start, 'days'), step=1)

    # mean daily value
    mean = ee.ImageCollection(days.map(lambda inday: data.filter(ee.Filter.calendarRange(inday, inday, 'day_of_year')).mean().set('year', i).set('day', ee.Number.int(inday))))
    regions = mean.map(lambda image: dailyRegions(image=image, descriptor=descriptor, scale=scale, projection=projection)).flatten()

    # export final table
    task = ee.batch.Export.table.toDrive(collection=regions,
                                         folder=export_folder,
                                         fileFormat='CSV',
                                         description=prefix + str(i),
                                         selectors=[shape_id, 'year', 'day', descriptor])
    task.start()

###
# Calculate relative humidity daily statistics,
###
def humidStatsByDay(descriptor, prefix, startYear, endYear, scale, projection):

  # Iterate time interval years.
  for i in range(startYear, endYear):

    # Select start and end string dates of the focal year,
    fyear_start = ee.Date(str(i) + '-01-01')
    fyear_end = ee.Date(str(i) + '-12-31')

    # select bands
    data = era5Collection.select(['temperature_2m','dewpoint_temperature_2m']).filter(ee.Filter.date(fyear_start, fyear_end)).map(relativeHumidity)

    # days for focal year (based on collection size)
    days = ee.List.sequence(start=1, end=fyear_end.difference(fyear_start, 'days'), step=1)

    # mean daily value
    mean = ee.ImageCollection(days.map(lambda inday: data.filter(ee.Filter.calendarRange(inday, inday, 'day_of_year')).mean().set('year', i).set('day', ee.Number.int(inday))))
    regions = mean.map(lambda image: dailyRegions(image=image, descriptor=descriptor, scale=scale, projection=projection)).flatten()

    # export final table
    task = ee.batch.Export.table.toDrive(collection=regions,
                                         folder=export_folder,
                                         fileFormat='CSV',
                                         description=prefix + str(i),
                                         selectors=[shape_id, 'year', 'day', descriptor])
    task.start()

###
# Calculate hourly statistics,
###
def statsByHour(descriptor, band, prefix, startYear, endYear, scale, projection):

  # Iterate time interval years.
  for i in range(startYear, endYear):

    # Select start and end string dates of the focal year, note the shift in dates
    fyear_start = ee.Date(str(i) + '-01-02')
    fyear_end = ee.Date(str(i+1) + '-01-01')

    # Select precipitation from collection,
    # filter for period of interest,
    # select cumulated precipitation,
    # multiply by 1000 and insert year and day (one day before because of shift in dates).
    data = era5Collection \
            .select(band) \
            .filter(ee.Filter.date(fyear_start, fyear_end)) \
            .filterMetadata('hour', 'equals', 0) \
            .map(lambda img: img.multiply(1000).set('year', i).set('day', ee.Date(img.get('system:time_start')).advance(-1, 'day').format('D')))

    # Extract means for regions.
    regions = data.map(lambda image: dailyRegions(image=image, descriptor=descriptor, scale=scale, projection=projection)).flatten()

    # export final table
    task = ee.batch.Export.table.toDrive(collection=regions,
                                         folder=export_folder,
                                         fileFormat='CSV',
                                         description=prefix + str(i),
                                         selectors=[shape_id, 'year', 'day', descriptor])
    task.start()

###
# Calculate wind daily statistics,
###
def statsWindByDay(descriptor, prefix, startYear, endYear, scale, projection):

  # Iterate time interval years.
  for i in range(startYear, endYear):

    # Select start and end string dates of the focal year,
    fyear_start = ee.Date(str(i) + '-01-01')
    fyear_end = ee.Date(str(i) + '-12-31')

    # Select band and calculate wind speed.
    data = era5Collection.filter(ee.Filter.date(fyear_start, fyear_end)).map(windSpeed)

    # days for focal year (based on collection size)
    days = ee.List.sequence(start=1, end=fyear_end.difference(fyear_start, 'days'), step=1)

    # mean daily value
    mean = ee.ImageCollection(days.map(lambda inday: data.filter(ee.Filter.calendarRange(inday, inday, 'day_of_year')).mean().set('year', i).set('day', inday)))
    regions = mean.map(lambda image: dailyRegions(image=image, descriptor=descriptor, scale=scale, projection=projection)).flatten()

    # export final table
    task = ee.batch.Export.table.toDrive(collection=regions,
                                         folder=export_folder,
                                         fileFormat='CSV',
                                         description=prefix + str(i),
                                         selectors=[shape_id, 'year', 'day', descriptor])
    task.start()

In [ ]:
# ERA 5 Globals.
startYear = 2000
endYear = 2023
scale = 1000
projection = 'EPSG:4326'

#### Temperature

In [ ]:
# Set locals.
band = 'temperature_2m'
descriptor = 'temperature_2m'
prefix = 'temperature_2m_'

# Get daily statistics.
statsByDay(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

#### Precipitation

In [ ]:
# Set locals.
descriptor = 'total_precipitation'
band = 'total_precipitation'
prefix = 'total_precipitation_'

# Get daily statistics.
statsByHour(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

#### Wind
The meteorological convention for winds is that U component is positive for a west to east flow (eastward wind) and the V component is positive for south to north flow
(northward wind).



In [ ]:
# Set globals.
descriptor = 'wind_speed'
prefix = 'wind_speed_'
projection = 'EPSG:4326'

# Get daily statistics from hourly data.
statsWindByDay(descriptor=descriptor, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

### Soil water content

In [ ]:
# layer 1 Layer 1: 0 - 7cm
descriptor = 'soil_water_0_7cm'
band = 'volumetric_soil_water_layer_1'
prefix = 'soil_water_7cm_'
statsByDay(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

In [ ]:
# Layer 2: 7 - 28cm
descriptor = 'soil_water_7_28cm'
band = 'volumetric_soil_water_layer_2'
prefix = 'soil_water_28cm_'
statsByDay(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

In [ ]:
# Layer 3: 28 - 100cm
descriptor = 'soil_water_28_100cm'
band = 'volumetric_soil_water_layer_3'
prefix = 'soil_water_100cm_'
statsByDay(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

In [ ]:
# Layer 4: 100 - 289cm
descriptor = 'soil_water_100_289cm'
band = 'volumetric_soil_water_layer_4'
prefix = 'soil_water_289cm_'
statsByDay(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

### Soil temperature

In [ ]:
# Layer 1: 0 - 7cm
descriptor = 'soil_temperature_0_7cm'
band = 'soil_temperature_level_1'
prefix = 'soil_temperature_7cm_'
statsByDay(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

In [ ]:
# Layer 2: 7 - 28cm
descriptor = 'soil_temperature_7_28cm'
band = 'soil_temperature_level_2'
prefix = 'soil_temperature_28cm_'
statsByDay(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

In [ ]:
# Layer 3: 28 - 100cm
descriptor = 'soil_temperature_28_100cm'
band = 'soil_temperature_level_3'
prefix = 'soil_temperature_100cm_'
statsByDay(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

In [ ]:
# Layer 4: 100 - 289cm
descriptor = 'soil_temperature_100_289cm'
band = 'soil_temperature_level_4'
prefix = 'soil_temperature_289cm_'
statsByDay(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

### Latent heat flux
surface_latent_heat_flux	J/m2
Exchange of latent heat with the surface through turbulent diffusion. This variables is accumulated from the beginning of the forecast time to the end of the forecast step. By model convention, downward fluxes are positive.


In [ ]:
# Latent heat flux.
descriptor = 'surface_latent_heat_flux'
band = 'surface_latent_heat_flux'
prefix = 'latent_heat_flux'
statsByDay(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

### Incoming solar radiation
surface_net_solar_radiation	J/m2
Amount of solar radiation (also known as shortwave radiation) reaching the surface of the Earth (both direct and diffuse) minus the amount reflected by the Earth's surface (which is governed by the albedo).Radiation from the Sun (solar, or shortwave, radiation) is partly reflected back to space by clouds and particles in the atmosphere (aerosols) and some of it is absorbed. The rest is incident on the Earth's surface, where some of it is reflected. The difference between downward and reflected solar radiation is the surface net solar radiation. This variable is accumulated from the beginning of the forecast time to the end of the forecast step. The units are joules per square metre (J m-2). To convert to watts per square metre (W m-2), the accumulated values should be divided by the accumulation period expressed in seconds. The ECMWF convention for vertical fluxes is positive downwards.


In [ ]:
# Incoming solar radiation.
descriptor = 'surface_net_solar_radiation'
band = 'surface_net_solar_radiation'
prefix = 'incoming_solar_radiation'
statsByDay(descriptor=descriptor, band=band, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)

### Relative humidity
Formula from https://en.wikipedia.org/wiki/Dew_point

In [ ]:
# Relative humidity.
descriptor = 'relative_humidity'
prefix = 'relative_humidity'
humidStatsByDay(descriptor=descriptor, prefix=prefix, startYear=startYear, endYear=endYear, scale=scale, projection=projection)